In [176]:
import hashlib
import copy


class Block:
    def __init__(self, transactions, previous_hash, timestamp=None, nonce=0, summary=None, hash=None,
                 merkle_root=None):
        self.timestamp = timestamp if timestamp is not None else datetime.now()
        self.transactions = transactions
        self.previous_hash = previous_hash
        self.nonce = nonce
        self.summary = summary if summary is not None else {}
        #self.validate_summary()
        self.hash = hash if hash is not None else self.calculate_hash()
        self.merkle_root = merkle_root if merkle_root is not None else self.calculate_merkle_root()

    def validate_summary(self):
        temp_summary = self.summary
        for transaction in self.transactions:
            temp_summary = self._update_summary(transaction, temp_summary)

        self.summary = temp_summary

    def calculate_hash(self):
        return hashlib.sha256((str(self.timestamp) + str(self.transactions) + str(self.previous_hash) + str(
            self.nonce)).encode()).hexdigest()

    def proof_of_work(self, difficulty):
        while self.hash[0:difficulty] != "0" * difficulty:
            self.nonce += 1
            self.hash = self.calculate_hash()

    def calculate_merkle_root(self):
        transaction_representations = [transaction.__repr__() for transaction in self.transactions]
        return get_merkle_tree_root(transaction_representations)

    def _update_summary(self, transaction, temp_summary):
        if transaction.receiver not in temp_summary:
            if transaction.receiver == transaction.sender:
                temp_summary[transaction.receiver] = [transaction.amount]
                return temp_summary
            else:
                raise Exception("Receiver account doesn't exist yet")

        if transaction.sender not in temp_summary:
            raise Exception("Sender account doesn't exist yet")

        if temp_summary[transaction.sender][-1] < transaction.amount and transaction.receiver != transaction.sender:
            raise Exception("Sender account doesn't have enough money")

        temp_summary[transaction.receiver].append(temp_summary[transaction.receiver][-1] + transaction.amount)
        if transaction.receiver != transaction.sender:
            temp_summary[transaction.sender].append(temp_summary[transaction.sender][-1] - transaction.amount)

        return temp_summary

    def print_summary(self):
        for key, value in self.summary.items():
            print("Person:", key)
            print("Amount history:", value)
            print("Max amount:", max(value))
            print("Min amount:", min(value))
            print()

In [177]:
def get_merkle_tree_root(hash_info):
    if type(hash_info[0]) == str:
        hashes = [hashlib.sha256(info.encode()).hexdigest() for info in hash_info]
    else:
        hashes = hash_info

    if len(hashes) == 1:
        return hashes[0]

    while len(hashes) % 2 != 0:
        hashes.append(hashes[-1])

    next_level_hashes = []
    for i in range(0, len(hashes), 2):
        combined_hash = hashes[i] + hashes[i + 1]
        new_hash = hashlib.sha256(combined_hash.encode()).hexdigest()
        next_level_hashes.append(new_hash)

    return get_merkle_tree_root(next_level_hashes)

In [178]:
test = ["Bob sent 10 to Alice",
        "Bob sent 60 to Alice",
        "Alice sent 10 to Tom"]

aaaa = get_merkle_tree_root(test)
aaaa

'e0ca05fa2a1984ae56ec4dd5db6def1b63a3b1ae6ef26fda3513a0a6b5ac4f4f'

In [179]:
class Blockchain:
    def __init__(self, difficulty, chain=None):
        self.chain = chain if chain is not None else [self._create_genesis_block()]
        self.difficulty = difficulty

    def _create_genesis_block(self):
        return Block([Transaction("Genesis", "Genesis", 0)], "0")

    def _get_latest_block(self):
        return self.chain[-1]

    def add_block(self, new_block):
        new_block.previous_hash = self._get_latest_block().hash
        new_block.proof_of_work(self.difficulty)
        new_block.summary = copy.deepcopy(self._get_latest_block().summary)
        new_block.validate_summary()

        self.chain.append(new_block)

    def is_chain_valid(self):
        for i in range(1, len(self.chain)):
            current_block = self.chain[i]
            previous_block = self.chain[i - 1]

            if current_block.hash != current_block.calculate_hash() or current_block.previous_hash != previous_block.hash:
                print(current_block.hash, current_block.calculate_hash(), current_block.previous_hash, previous_block.hash)
                print("Invalid block hash\n")
                return False

            if current_block.merkle_root != current_block.calculate_merkle_root():
                print("Invalid block merkle root\n")
                return False

        print("Blockchain is valid\n")
        return True

    def print_chain(self):
        for block in self.chain:
            print("Timestamp:", block.timestamp)
            print("Transactions:", block.transactions)
            print("Nonce:", block.nonce)
            print("Merkle Root:", block.merkle_root)
            print("Hash:", block.hash)
            print()

In [180]:
class Transaction:
    def __init__(self, sender, receiver, amount):
        self.sender = sender
        self.receiver = receiver
        self.amount = amount

    def __repr__(self):
        if self.amount == int(self.amount):
            amount_str = f'{self.amount:.1f}'
        else:
            amount_str = f'{self.amount}'
        return f'{self.sender} sent {amount_str} to {self.receiver}'

In [181]:
from sqlalchemy import create_engine, Column, Integer, String, Float, DateTime, JSON, ForeignKey
from sqlalchemy.orm import relationship, sessionmaker
from sqlalchemy.orm import declarative_base
from datetime import datetime

Base = declarative_base()


class TransactionTable(Base):
    __tablename__ = 'transactions'

    id = Column(Integer, primary_key=True)
    sender = Column(String, nullable=False)
    receiver = Column(String, nullable=False)
    amount = Column(Float, nullable=False)
    block_id = Column(Integer, ForeignKey('blocks.id'))

    block = relationship("BlockTable", back_populates="transactions")


class BlockTable(Base):
    __tablename__ = 'blocks'

    id = Column(Integer, primary_key=True)
    timestamp = Column(DateTime, nullable=False)
    previous_hash = Column(String, nullable=False)
    nonce = Column(Integer, nullable=False)
    hash = Column(String, nullable=False)
    merkle_root = Column(String, nullable=False)
    summary = Column(JSON, nullable=True)

    transactions = relationship("TransactionTable", back_populates="block")
    blockchain_id = Column(Integer, ForeignKey('blockchains.id'))

    blockchain = relationship("BlockchainTable", back_populates="blocks")


class BlockchainTable(Base):
    __tablename__ = 'blockchains'

    id = Column(Integer, primary_key=True)
    difficulty = Column(Integer, nullable=False)

    blocks = relationship("BlockTable", back_populates="blockchain")


engine = create_engine('')
Base.metadata.create_all(engine)
Session = sessionmaker(bind=engine)


def save_blockchain(blockchain):
    session = Session()

    try:
        blockchain_table = BlockchainTable(difficulty=blockchain.difficulty)
        session.add(blockchain_table)
        session.flush()

        for block in blockchain.chain:
            block_table = BlockTable(
                timestamp=block.timestamp,
                previous_hash=block.previous_hash,
                nonce=block.nonce,
                hash=block.hash,
                merkle_root=block.merkle_root,
                summary=block.summary,
                blockchain_id=blockchain_table.id
            )
            session.add(block_table)
            session.flush()

            for transaction in block.transactions:
                transaction_table = TransactionTable(
                    sender=transaction.sender,
                    receiver=transaction.receiver,
                    amount=transaction.amount,
                    block_id=block_table.id
                )
                session.add(transaction_table)

        session.commit()
        print("Blockchain saved successfully!")

    except Exception as e:
        session.rollback()
        print("Error saving blockchain:", e)

    finally:
        session.close()


def load_blockchain_by_id(blockchain_id):
    session = Session()
    try:

        blockchain_table = session.query(BlockchainTable).filter_by(id=blockchain_id).first()

        if not blockchain_table:
            print(f"Blockchain with id {blockchain_id} not found in the database.")
            return None

        chain = []

        blocks = session.query(BlockTable).filter_by(blockchain_id=blockchain_table.id).all()

        for block_table in blocks:
            transactions = []

            block_transactions = session.query(TransactionTable).filter_by(block_id=block_table.id).all()

            for transaction_table in block_transactions:
                transaction = Transaction(sender=transaction_table.sender,
                                          receiver=transaction_table.receiver,
                                          amount=transaction_table.amount)
                transactions.append(transaction)

            block = Block(transactions=transactions,
                          previous_hash=block_table.previous_hash,
                          timestamp=block_table.timestamp,
                          nonce=block_table.nonce,
                          hash=block_table.hash,
                          summary=block_table.summary,
                          merkle_root=block_table.merkle_root)
            chain.append(block)
        blockchain = Blockchain(difficulty=blockchain_table.difficulty, chain=chain)

        print(f"Blockchain with id {blockchain_id} loaded successfully!")
        return blockchain

    except Exception as e:
        print("Error loading blockchain from the database:", e)
        return None

    finally:
        session.close()


In [182]:
blockchain = Blockchain(4)
transactions = [Transaction("Alice", "Alice", 100),
                Transaction("Alice", "Alice", 200),
                Transaction("Bob", "Bob", 500),
                Transaction("Charlie", "Charlie", 400),
                Transaction("Alice", "Bob", 10),
                Transaction("Bob", "Charlie", 5),
                Transaction("Charlie", "Alice", 3)]

block1 = Block(transactions, "0")
try:
    blockchain.add_block(block1)
except Exception as e:
    print(e, "\n")

transactions = [Transaction("Tom", "Alice", 10000)]
invalid_block = Block(transactions, "0")

try:
    blockchain.add_block(invalid_block)
except Exception as e:
    print(e, "\n")

transactions = [Transaction("Tom", "Tom", 1000),
                Transaction("Bob", "Bob", 200),
                Transaction("Tom", "Charlie", 10),
                Transaction("Charlie", "Bob", 50), ]

block2 = Block(transactions, "0")
try:
    blockchain.add_block(block2)
except Exception as e:
    print(e, "\n")

print("Blockchain blocks' information:")
blockchain.print_chain()

blockchain.is_chain_valid()

print("Block 1 amount history:")
block1.print_summary()

print("Block 2 amount history:")
block2.print_summary()

save_blockchain(blockchain)

Sender account doesn't exist yet 
Blockchain blocks' information:
Timestamp: 2024-05-23 00:17:20.744561
Transactions: [Genesis sent 0.0 to Genesis]
Nonce: 0
Merkle Root: c6335cf7be0965ca24f3d43ebfd03dad3eee5a667e33b4e48ab79ed28f5b501c
Hash: 17e10e891c90ce4add8b7c547cf6a2965a7e75edb1ea5b4a2ae9e6d2487b8df9

Timestamp: 2024-05-23 00:17:20.744561
Transactions: [Alice sent 100.0 to Alice, Alice sent 200.0 to Alice, Bob sent 500.0 to Bob, Charlie sent 400.0 to Charlie, Alice sent 10.0 to Bob, Bob sent 5.0 to Charlie, Charlie sent 3.0 to Alice]
Nonce: 116673
Merkle Root: 9ae80755a6850c79272eb07556e201701b1d1d6f2e71541d7edbb3f917825e80
Hash: 0000e5c89958fb0244b62b63d890334db11ac0b87c94012e3bbba7cbd3b846d7

Timestamp: 2024-05-23 00:17:21.559025
Transactions: [Tom sent 1000.0 to Tom, Bob sent 200.0 to Bob, Tom sent 10.0 to Charlie, Charlie sent 50.0 to Bob]
Nonce: 291968
Merkle Root: 7de87835693155b72fe7faa41426eb00b6a2c1d47cef05b99d30e6429c31be43
Hash: 00003672988af8d2700b36c338e494014b2e689d2c

In [183]:
blockchain = load_blockchain_by_id(1)

Blockchain with id 1 loaded successfully!


In [184]:
blockchain.is_chain_valid()
blockchain.print_chain()
blockchain.chain[1].print_summary()

Blockchain is valid

Timestamp: 2024-05-23 00:17:20.744561
Transactions: [Genesis sent 0.0 to Genesis]
Nonce: 0
Merkle Root: c6335cf7be0965ca24f3d43ebfd03dad3eee5a667e33b4e48ab79ed28f5b501c
Hash: 17e10e891c90ce4add8b7c547cf6a2965a7e75edb1ea5b4a2ae9e6d2487b8df9

Timestamp: 2024-05-23 00:17:20.744561
Transactions: [Alice sent 100.0 to Alice, Alice sent 200.0 to Alice, Bob sent 500.0 to Bob, Charlie sent 400.0 to Charlie, Alice sent 10.0 to Bob, Bob sent 5.0 to Charlie, Charlie sent 3.0 to Alice]
Nonce: 116673
Merkle Root: 9ae80755a6850c79272eb07556e201701b1d1d6f2e71541d7edbb3f917825e80
Hash: 0000e5c89958fb0244b62b63d890334db11ac0b87c94012e3bbba7cbd3b846d7

Timestamp: 2024-05-23 00:17:21.559025
Transactions: [Tom sent 1000.0 to Tom, Bob sent 200.0 to Bob, Tom sent 10.0 to Charlie, Charlie sent 50.0 to Bob]
Nonce: 291968
Merkle Root: 7de87835693155b72fe7faa41426eb00b6a2c1d47cef05b99d30e6429c31be43
Hash: 00003672988af8d2700b36c338e494014b2e689d2c1d2a98f927e87aee7a2d65

Person: Alice
Amount 

In [185]:
blockchain.chain[1].print_summary()

Person: Alice
Amount history: [100, 300, 290, 293]
Max amount: 300
Min amount: 100

Person: Bob
Amount history: [500, 510, 505]
Max amount: 510
Min amount: 500

Person: Charlie
Amount history: [400, 405, 402]
Max amount: 405
Min amount: 400
